<a href="https://colab.research.google.com/github/anagha-h/PLAI-Project/blob/main/RAGPLAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLM Setup


---



In [9]:
!pip install pandas sentence-transformers pandarallel faiss-cpu

In [10]:
!pip install transformers accelerate torch

In [11]:
!pip install --upgrade transformers huggingface_hub

In [ ]:
!huggingface-cli login --token ""

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `GoogleCollabRag2.0` has been saved to /home/nagarw48/.cache/huggingface/stored_tokens
Traceback (most recent call last):
  File "/home/nagarw48/Projects/PLAI/PLAI-Project-3/.venv/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/home/nagarw48/Projects/PLAI/PLAI-Project-3/.venv/lib64/python3.9/site-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "/home/nagarw48/Projects/PLAI/PLAI-Project-3/.venv/lib64/python3.9/site-packages/huggingface_hub/commands/user.py", line 153, in run
    login(
  File "/home/nagarw48/Projects/PLAI/PLAI-Project-3/.venv/lib64/python3.9/site-packages/huggingface_hub/_login.py", line 119, in login
    _login(token, add_t

In [13]:
# !pip install --upgrade transformers

In [14]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
# #llama2
# import requests
# import torch
# from PIL import Image
# from transformers import LlamaForCausalLM, LlamaTokenizer
# import transformers

# # Loading LLAMA 2 Model and Tokenizer
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Loading LLAMA 2 model...")

# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# print("Model loaded successfully!")

# llama_model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
# # llama 2
# from transformers import pipeline

# llama_pipeline = pipeline(
#     "text-generation",  # LLM task
#     model=model_name,
#     torch_dtype=torch.float16,
#     #device_map="auto",
#     device= "cuda",
# )

In [16]:
# #llama3 vision
# import requests
# import torch
# from PIL import Image
# from transformers import MllamaForConditionalGeneration, AutoProcessor

# model_id = "meta-llama/Llama-3.2-11B-Vision"

# model = MllamaForConditionalGeneration.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# processor = AutoProcessor.from_pretrained(model_id)

In [17]:
# vision 3 instruct
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
model = MllamaForConditionalGeneration.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.74it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


RAG Model


In [18]:
import torch
from transformers import CLIPProcessor, CLIPModel, LlamaTokenizer, LlamaForCausalLM
import faiss
import numpy as np
import json
from PIL import Image
import io
import requests
#from VectorDBPLAI import image_bank

In [20]:
import os

# Set current working directory to the script's location
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

# Load FAISS index and metadata
faiss_index = faiss.read_index("vector_db_f.faiss")
with open("metadata_store_f.json", "r") as f:
    metadata_store = json.load(f)


In [21]:
# Load CLIP model
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

In [22]:
def encode_image(image_path):
    # Load the image from file path
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    image_input = clip_processor(images=image, return_tensors="pt")
    pixel_values = image_input['pixel_values']

    # Generate embedding using CLIP model
    with torch.no_grad():
         embedding = clip_model.get_image_features(pixel_values)

    # Close the image file
    image.close()

    return embedding

In [24]:
# Function to retrieve relevant documents and images
def retrieve(image_embedding, top_k=5):
    #retrieve
    results =[]
    distances, indices = faiss_index.search(image_embedding, top_k)
    relevant_images = []
    for i, idx in enumerate(indices[0]):
      if idx in metadata_store:
        relevant_images.append(image_bank[idx])
        results.append(
              {"document_id": metadata_store[idx]["document_id"],
              "page_num": metadata_store[idx]["page_num"],
              "img_index": metadata_store[idx]["img_index"],
              "content": metadata_store[idx]["content"],
              "distance": distances[i]}
        )
    context = " ".join([item["content"] for item in results])
    return context

In [25]:
def generate_instruct(context, query_image_path, query_text):
    image = Image.open(query_image_path)
    #coordinates = {"Start position: (1, 2), Goal position: (3, 1), Obstacle positions: (1, 0) (1, 1), (3, 3)"}
    messages = [
        {
            "role": "system",
            "content": f'''You are an AI agent operating in a grid world environment.
            Your task is to navigate from a given starting point to a goal point while avoiding obstacles and finding an optimal path.
            The grid is defined as follows:
            - The top left cell is indexed as (0, 0), the cell down to it is indexed (1, 0), right to it is (0, 1).
            - The indexing of row and columns start from 0.
            - The starting point is represented by a blue colored box labeled "R", which marks your initial position.
            - The goal point is represented by a red colored box labeled "G", indicating where you need to reach.
            - The grid contains obstacles, shown as black colored boxes, which you are not allowed to go to it any step. Instead you must find a way around using white cells to reach the goal.
            - Don't assume obstacle, goal or the start positions only consider those given to you in {query_image_path}.
            - You can only move one cell at a time, you cannot jump over cells.
            - The grid is made up of heavy black colored horizontal and vertical lines, which make up the rows and columns.
            - After you reach the Goal coordinates, stop immediately, your task is complete.

            You can move in the grid in the following directions:
            - Right (the coordinates are : row, column+1),
            - Left (the coordinates are: row, column-1),
            - Up (the coordinates are: row+1, column),
            - Down (the coordinates are: row-1, column).

            Your objective is to plan an efficient path from the starting point to the goal point while avoiding all obstacles.
            The size of the grid where number of rows =  number of columns (rows*columns), the positions of the starting point, goal point, and obstacles can be inferred from the input image which is <|image|> provided by the user
            Additionally, the provided context which is {context} which includes gridworld description in text format and its similar solutions respectively, to aid in your planning process.
            Start by identifying the number of rows and columns. The coordinates more than or equal to the number of rows is out of bounds. The coordinates more than or equal to the number of columns is out of bounds. The coordinates less than 0 are out of bounds. 
    

            Use the image and context collaboratively to generate a detailed and optimal navigation plan that successfully guides you to the goal.'''},
           {
              "role": "user",
              "content": [query_image_path, query_text]
           },
    ]
    # Generate tokenized text
    text = processor.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        return_tensors="pt"
    )
    
    # Decode tokens to a string
    decoded_text = processor.decode(text[0])
    
    # Process image and text together
    inputs = processor(image, decoded_text, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=1000)
    return processor.decode(output[0])

In [26]:
# def generate_vision3(query_image_path):
#     image = Image.open(query_image_path)
#     prompt = "<|image|><|begin_of_text|>\nYou are an AI agent operating in a grid world environment.\nYour task is to navigate from a given starting point to a goal point while avoiding obstacles and finding an optimal path.\nThe grid is defined as follows:\n- The starting point is represented by a blue colored box labeled ""R"", which marks your initial position.\n- The goal point is represented by a red colored box labeled ""G"", indicating where you need to reach.\n- The grid contains hurdles, shown as black colored boxes, which you must avoid during navigation.- The grid is made up of heavy black colored horizontal and vertical lines, which make up the rows and columns.\nYou can move in the grid in the following directions:\n- Right (move to the adjacent cell on the right),\n- Left (move to the adjacent cell on the left),\n- Up (move to the adjacent cell above),\n- Down (move to the adjacent cell below).\nYour objective is to plan an efficient path from the starting point to the goal point while avoiding all hurdles.\nThe size of the grid (rows*columns), the positions of the starting point, goal point, and hurdles can be inferred from the input image which is {query_image_path} provided by the user\nAdditionally, the provided context which is {context} includes gridworld solutions, to aid in your planning process.\nUse the image and context collaboratively to generate a detailed and optimal navigation plan that successfully guides you to the goal."
#     inputs = processor(image, prompt, return_tensors="pt").to(model.device)
#     output = model.generate(**inputs)
#     return processor.decode(output[0])

In [ ]:
# def generate_llama2_2(query_image_path, context, model, processor):
#     """
#     Generate a response based on a query image and the provided context.

#     Parameters:
#         query_image_path (str): Path to the input image for the query.
#         context (str): The context text retrieved using relevant documents/images.
#         model (transformers.PreTrainedModel): The model used for generating responses (e.g., Llama).
#         processor (transformers.PreTrainedProcessor): The processor used for preparing inputs (e.g., CLIP or Llama processor).

#     Returns:
#         str: The generated response from the model.
#     """
#     from PIL import Image

#     # Load the query image and preprocess it using the processor
#     image = Image.open(query_image_path)

#     # Create the prompt by including both the image and context
#     prompt = (
#         f"<|image|><|begin_of_text|>\n"
#         f"You are an AI agent operating in a grid world environment.\n"
#         f"Your task is to navigate from a given starting point to a goal point while avoiding obstacles and finding an optimal path.\n"
#         f"The grid is defined as follows:\n"
#         f"- The starting point is represented by a blue colored box labeled 'R', which marks your initial position.\n"
#         f"- The goal point is represented by a red colored box labeled 'G', indicating where you need to reach.\n"
#         f"- The grid contains hurdles, shown as black colored boxes, which you must avoid during navigation.\n"
#         f"- The grid is made up of heavy black colored horizontal and vertical lines, which make up the rows and columns.\n"
#         f"You can move in the grid in the following directions:\n"
#         f"- Right (move to the adjacent cell on the right),\n"
#         f"- Left (move to the adjacent cell on the left),\n"
#         f"- Up (move to the adjacent cell above),\n"
#         f"- Down (move to the adjacent cell below).\n"
#         f"Your objective is to plan an efficient path from the starting point to the goal point while avoiding all hurdles.\n"
#         f"The size of the grid (rows*columns), the positions of the starting point, goal point, and hurdles can be inferred from the input image which is {query_image_path} provided by the user.\n"
#         f"Additionally, the provided context which is {context} includes gridworld solutions, to aid in your planning process.\n"
#         f"Use the image and context collaboratively to generate a detailed and optimal navigation plan that successfully guides you to the goal."
#     )
    
    

#     # tokenizer = AutoTokenizer.from_pretrained("Llama-3.2-11B-Vision")
#     # config = AutoConfig.from_pretrained('Llama-3.2-11B-Vision')

#     # tokenizer.save_pretrained(os.getcwd())
#     # config.save_pretrained(os.getcwd())

#     # tokenizer = AutoTokenizer.from_pretrained(os.getcwd())
    
    
#     prompt_tokenize = tokenizer(prompt, return_tensors="pt")

#     # Process both the image and the prompt text together
#     inputs = processor(images=image, text=prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

#     # Generate the output from the model with controlled max_new_tokens
#     output = model.generate(
#         **inputs,
#         max_new_tokens=500,  # Adjust the number of tokens to generate
#         # temperature=0.7,  # Control randomness
#         # top_p=0.9,  # Use nucleus sampling
#         # repetition_penalty=1.1  # Avoid repeating outputs
#     )

#     # Decode and return the generated response
#     return processor.decode(output[0], skip_special_tokens=True)

In [21]:
# #generate
# def generate_llama2(context, image_embedding):

#     messages = [
#         {
#             "role": "system",
#             "content": f'''You are an AI agent operating in a grid world environment.
#             Your task is to navigate from a given starting point to a goal point while avoiding obstacles and finding an optimal path.
#             The grid is defined as follows:
#             - The topmost left cell is indexed as 0, Y0.
#             - The starting point is represented by a blue colored box labeled "R", which marks your initial position.
#             - The goal point is represented by a red colored box labeled "G", indicating where you need to reach.
#             - The grid contains hurdles, shown as black colored boxes, which you must avoid during navigation.
#             - The grid is made up of heavy black colored horizontal and vertical lines, which make up the rows and columns.

#             You can move in the grid in the following directions:
#             - Right (move to the adjacent cell on the right),
#             - Left (move to the adjacent cell on the left),
#             - Up (move to the adjacent cell above),
#             - Down (move to the adjacent cell below).

#             Your objective is to plan an efficient path from the starting point to the goal point while avoiding all hurdles.
#             The size of the grid (rows*columns), the positions of the starting point, goal point, and hurdles can be inferred from the input image which is {query_image_path} provided by the user
#             Additionally, the provided context which is {context} includes gridworld respectively, to aid in your planning process.

#             Use the image and context collaboratively to generate a detailed and optimal navigation plan that successfully guides you to the goal.'''},
#            {
#               "role": "user",
#               "content": query_image_path
#            },
#     ]

#     sequences = llama_pipeline(
#         messages,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length= 900,
#     )
#     return (sequences[0]['generated_text'])

In [27]:
if __name__ == "__main__":

    # Set current working directory to the script's location
    # os.chdir(os.path.dirname(os.path.abspath(__file__)))


    # Path to the query image
    query_image_path = "testData/images/gridworld_20241124_190937.png"
    query_text="Describe the given image and give me the path from robot R (in blue) to the goal G (in red) while avoiding the obstacles (black color filled boxed)."

    image_embedding = encode_image(query_image_path)
    result = retrieve(image_embedding, top_k=3)

    generated_output = generate_instruct(result, query_image_path, query_text)

    # Print the generated response
    print("Generated Response:")
    print(generated_output)

Generated Response:
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Dec 2024

You are an AI agent operating in a grid world environment.
            Your task is to navigate from a given starting point to a goal point while avoiding obstacles and finding an optimal path.
            The grid is defined as follows:
            - The top left cell is indexed as (0, 0), the cell down to it is indexed (1, 0), right to it is (0, 1).
            - The indexing of row and columns start from 0.
            - The starting point is represented by a blue colored box labeled "R", which marks your initial position.
            - The goal point is represented by a red colored box labeled "G", indicating where you need to reach.
            - The grid contains obstacles, shown as black colored boxes, which you are not allowed to go to it any step. Instead you must find a way around using white cells to reach the goal.
